In [1]:
import json
import pyspark as ps
from pyspark.sql.types import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

plt.rcParams['figure.figsize'] = [30, 15]


In [9]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("Capstone 1") \
            .config("spark.sql.caseSensitive", "true")\
            .getOrCreate()
sc = spark.sparkContext
sc

<SparkContext master=local[4] appName=Capstone 1>

In [27]:
import re

structure = r'' # tmp_df.value.substr([position],[length]).alias('colname')
last_loc = -1
with open('data/jan10dd.txt', 'r') as d:
        data_dict = d.readlines()

for line in data_dict:
    header = re.findall("\A([A-Z0-9]*)\s", line)
    loc = re.findall("(\d+)\s*-\s*(\d+)$", line)
    size = re.findall("\A[A-Z0-9]+\s+(\d+)", line)
    if header != []:
        if size != []:
            if loc != []: # and int(loc[0][1])>last_loc and int(loc[0][1])<1900:
#                 structure[header] = f"tmp_df.value.substr({int(loc[0][0])}, {int(size[0])}).alias('{header}')"
                if header[0] == 'FILLER':
                    header[0] = f'FILLER{int(loc[0][0])}'
                if header[0] == 'PADDING':
                    header[0] = f'PADDING{int(loc[0][0])}'
                structure += f"tmp_df.value.substr({int(loc[0][0]):>2}, {int(size[0]):>2}).alias('{header[0]}'), "
                last_loc = int(loc[0][0])

structure


"tmp_df.value.substr( 1, 15).alias('HRHHID'), tmp_df.value.substr(16,  2).alias('HRMONTH'), tmp_df.value.substr(18,  4).alias('HRYEAR4'), tmp_df.value.substr(22,  2).alias('HURESPLI'), tmp_df.value.substr(24,  3).alias('HUFINAL'), tmp_df.value.substr(27,  2).alias('HUSPNISH'), tmp_df.value.substr(29,  2).alias('HETENURE'), tmp_df.value.substr(31,  2).alias('HEHOUSUT'), tmp_df.value.substr(33,  2).alias('HETELHHD'), tmp_df.value.substr(35,  2).alias('HETELAVL'), tmp_df.value.substr(37,  2).alias('HEPHONEO'), tmp_df.value.substr(39,  2).alias('HEFAMINC'), tmp_df.value.substr(41,  2).alias('HUTYPEA'), tmp_df.value.substr(43,  2).alias('HUTYPB'), tmp_df.value.substr(47, 10).alias('HWHHWGT'), tmp_df.value.substr(57,  2).alias('HRINTSTA'), tmp_df.value.substr(59,  2).alias('HRNUMHOU'), tmp_df.value.substr(61,  2).alias('HRHTYPE'), tmp_df.value.substr(63,  2).alias('HRMIS'), tmp_df.value.substr(65,  2).alias('HUINTTYP'), tmp_df.value.substr(67,  2).alias('HUPRSCNT'), tmp_df.value.substr(71,  

In [29]:
tmp_df = spark.read.text('data/dec10pub.dat')

dec10_df = tmp_df.select(tmp_df.value.substr( 1, 15).alias('HRHHID'), tmp_df.value.substr(16,  2).alias('HRMONTH'), tmp_df.value.substr(18,  4).alias('HRYEAR4'), tmp_df.value.substr(22,  2).alias('HURESPLI'), tmp_df.value.substr(24,  3).alias('HUFINAL'), tmp_df.value.substr(27,  2).alias('HUSPNISH'), tmp_df.value.substr(29,  2).alias('HETENURE'), tmp_df.value.substr(31,  2).alias('HEHOUSUT'), tmp_df.value.substr(33,  2).alias('HETELHHD'), tmp_df.value.substr(35,  2).alias('HETELAVL'), tmp_df.value.substr(37,  2).alias('HEPHONEO'), tmp_df.value.substr(39,  2).alias('HEFAMINC'), tmp_df.value.substr(41,  2).alias('HUTYPEA'), tmp_df.value.substr(43,  2).alias('HUTYPB'), tmp_df.value.substr(47, 10).alias('HWHHWGT'), tmp_df.value.substr(57,  2).alias('HRINTSTA'), tmp_df.value.substr(59,  2).alias('HRNUMHOU'), tmp_df.value.substr(61,  2).alias('HRHTYPE'), tmp_df.value.substr(63,  2).alias('HRMIS'), tmp_df.value.substr(65,  2).alias('HUINTTYP'), tmp_df.value.substr(67,  2).alias('HUPRSCNT'), tmp_df.value.substr(71,  5).alias('HRHHID2'), tmp_df.value.substr(76,  3).alias('FILLER76'), tmp_df.value.substr(79,  2).alias('HUBUS'), tmp_df.value.substr(81,  2).alias('HUBUSL1'), tmp_df.value.substr(83,  2).alias('HUBUSL2'), tmp_df.value.substr(85,  2).alias('HUBUSL3'), tmp_df.value.substr(87,  2).alias('HUBUSL4'), tmp_df.value.substr(89,  2).alias('GEREG'), tmp_df.value.substr(91,  2).alias('GESTCEN'), tmp_df.value.substr(93,  2).alias('GESTFIPS'), tmp_df.value.substr(95,  1).alias('FILLER95'), tmp_df.value.substr(96,  5).alias('GTCBSA'), tmp_df.value.substr(101,  3).alias('GTCO'), tmp_df.value.substr(104,  1).alias('GTCBSAST'), tmp_df.value.substr(105,  1).alias('GTMETSTA'), tmp_df.value.substr(106,  1).alias('GTINDVPC'), tmp_df.value.substr(107,  1).alias('GTCBSASZ'), tmp_df.value.substr(108,  3).alias('GTCSA'), tmp_df.value.substr(111,  3).alias('FILLER111'), tmp_df.value.substr(114,  2).alias('PROLDRRP'), tmp_df.value.substr(116,  2).alias('PUPELIG'), tmp_df.value.substr(118,  2).alias('PERRP'), tmp_df.value.substr(120,  2).alias('PEPARENT'), tmp_df.value.substr(122,  2).alias('PEAGE'), tmp_df.value.substr(124,  1).alias('PRTFAGE'), tmp_df.value.substr(125,  2).alias('PEMARITL'), tmp_df.value.substr(127,  2).alias('PESPOUSE'), tmp_df.value.substr(129,  2).alias('PESEX'), tmp_df.value.substr(131,  2).alias('PEAFEVER'), tmp_df.value.substr(133,  2).alias('FILLER133'), tmp_df.value.substr(135,  2).alias('PEAFNOW'), tmp_df.value.substr(137,  2).alias('PEEDUCA'), tmp_df.value.substr(139,  2).alias('PTDTRACE'), tmp_df.value.substr(141,  2).alias('PRDTHSP'), tmp_df.value.substr(143,  2).alias('PUCHINHH'), tmp_df.value.substr(145,  2).alias('FILLER145'), tmp_df.value.substr(147,  2).alias('PULINENO'), tmp_df.value.substr(149,  2).alias('FILLER149'), tmp_df.value.substr(151,  2).alias('PRFAMNUM'), tmp_df.value.substr(153,  2).alias('PRFAMREL'), tmp_df.value.substr(155,  2).alias('PRFAMTYP'), tmp_df.value.substr(157,  2).alias('PEHSPNON'), tmp_df.value.substr(159,  2).alias('PRMARSTA'), tmp_df.value.substr(161,  2).alias('PRPERTYP'), tmp_df.value.substr(163,  3).alias('PENATVTY'), tmp_df.value.substr(166,  3).alias('PEMNTVTY'), tmp_df.value.substr(169,  3).alias('PEFNTVTY'), tmp_df.value.substr(172,  2).alias('PRCITSHP'), tmp_df.value.substr(174,  2).alias('PRCITFLG'), tmp_df.value.substr(176,  2).alias('PRINUSYR'), tmp_df.value.substr(178,  2).alias('PUSLFPRX'), tmp_df.value.substr(180,  2).alias('PEMLR'), tmp_df.value.substr(182,  2).alias('PUWK'), tmp_df.value.substr(184,  2).alias('PUBUS1'), tmp_df.value.substr(186,  2).alias('PUBUS2OT'), tmp_df.value.substr(188,  2).alias('PUBUSCK1'), tmp_df.value.substr(190,  2).alias('PUBUSCK2'), tmp_df.value.substr(192,  2).alias('PUBUSCK3'), tmp_df.value.substr(194,  2).alias('PUBUSCK4'), tmp_df.value.substr(196,  2).alias('PURETOT'), tmp_df.value.substr(198,  2).alias('PUDIS'), tmp_df.value.substr(200,  2).alias('PERET1'), tmp_df.value.substr(202,  2).alias('PUDIS1'), tmp_df.value.substr(204,  2).alias('PUDIS2'), tmp_df.value.substr(206,  2).alias('PUABSOT'), tmp_df.value.substr(208,  2).alias('PULAY'), tmp_df.value.substr(210,  2).alias('PEABSRSN'), tmp_df.value.substr(212,  2).alias('PEABSPDO'), tmp_df.value.substr(214,  2).alias('PEMJOT'), tmp_df.value.substr(216,  2).alias('PEMJNUM'), tmp_df.value.substr(218,  2).alias('PEHRUSL1'), tmp_df.value.substr(220,  2).alias('PEHRUSL2'), tmp_df.value.substr(222,  2).alias('PEHRFTPT'), tmp_df.value.substr(224,  3).alias('PEHRUSLT'), tmp_df.value.substr(227,  2).alias('PEHRWANT'), tmp_df.value.substr(229,  2).alias('PEHRRSN1'), tmp_df.value.substr(231,  2).alias('PEHRRSN2'), tmp_df.value.substr(233,  2).alias('PEHRRSN3'), tmp_df.value.substr(235,  2).alias('PUHROFF1'), tmp_df.value.substr(237,  2).alias('PUHROFF2'), tmp_df.value.substr(239,  2).alias('PUHROT1'), tmp_df.value.substr(241,  2).alias('PUHROT2'), tmp_df.value.substr(243,  2).alias('PEHRACT1'), tmp_df.value.substr(245,  2).alias('PEHRACT2'), tmp_df.value.substr(247,  3).alias('PEHRACTT'), tmp_df.value.substr(250,  2).alias('PEHRAVL'), tmp_df.value.substr(252,  5).alias('FILLER252'), tmp_df.value.substr(257,  2).alias('PUHRCK1'), tmp_df.value.substr(259,  2).alias('PUHRCK2'), tmp_df.value.substr(261,  2).alias('PUHRCK3'), tmp_df.value.substr(263,  2).alias('PUHRCK4'), tmp_df.value.substr(265,  2).alias('PUHRCK5'), tmp_df.value.substr(267,  2).alias('PUHRCK6'), tmp_df.value.substr(269,  2).alias('PUHRCK7'), tmp_df.value.substr(271,  2).alias('PUHRCK12'), tmp_df.value.substr(273,  2).alias('PULAYDT'), tmp_df.value.substr(275,  2).alias('PULAY6M'), tmp_df.value.substr(277,  2).alias('PELAYAVL'), tmp_df.value.substr(279,  2).alias('PULAYAVR'), tmp_df.value.substr(281,  2).alias('PELAYLK'), tmp_df.value.substr(283,  3).alias('PELAYDUR'), tmp_df.value.substr(286,  2).alias('PELAYFTO'), tmp_df.value.substr(288,  2).alias('PULAYCK1'), tmp_df.value.substr(290,  2).alias('PULAYCK2'), tmp_df.value.substr(292,  2).alias('PULAYCK3'), tmp_df.value.substr(294,  2).alias('PULK'), tmp_df.value.substr(296,  2).alias('PELKM1'), tmp_df.value.substr(298,  2).alias('PULKM2'), tmp_df.value.substr(300,  2).alias('PULKM3'), tmp_df.value.substr(302,  2).alias('PULKM4'), tmp_df.value.substr(304,  2).alias('PULKM5'), tmp_df.value.substr(306,  2).alias('PULKM6'), tmp_df.value.substr(308,  2).alias('PULKDK1'), tmp_df.value.substr(310,  2).alias('PULKDK2'), tmp_df.value.substr(312,  2).alias('PULKDK3'), tmp_df.value.substr(314,  2).alias('PULKDK4'), tmp_df.value.substr(316,  2).alias('PULKDK5'), tmp_df.value.substr(318,  2).alias('PULKDK6'), tmp_df.value.substr(320,  2).alias('PULKPS1'), tmp_df.value.substr(322,  2).alias('PULKPS2'), tmp_df.value.substr(324,  2).alias('PULKPS3'), tmp_df.value.substr(326,  2).alias('PULKPS4'), tmp_df.value.substr(328,  2).alias('PULKPS5'), tmp_df.value.substr(330,  2).alias('PULKPS6'), tmp_df.value.substr(332,  2).alias('PELKAVL'), tmp_df.value.substr(334,  2).alias('PULKAVR'), tmp_df.value.substr(336,  2).alias('PELKLL1O'), tmp_df.value.substr(338,  2).alias('PELKLL2O'), tmp_df.value.substr(340,  2).alias('PELKLWO'), tmp_df.value.substr(342,  3).alias('PELKDUR'), tmp_df.value.substr(345,  2).alias('PELKFTO'), tmp_df.value.substr(347,  2).alias('PEDWWNTO'), tmp_df.value.substr(349,  2).alias('PEDWRSN'), tmp_df.value.substr(351,  2).alias('PEDWLKO'), tmp_df.value.substr(353,  2).alias('PEDWWK'), tmp_df.value.substr(355,  2).alias('PEDW4WK'), tmp_df.value.substr(357,  2).alias('PEDWLKWK'), tmp_df.value.substr(359,  2).alias('PEDWAVL'), tmp_df.value.substr(361,  2).alias('PEDWAVR'), tmp_df.value.substr(363,  2).alias('PUDWCK1'), tmp_df.value.substr(365,  2).alias('PUDWCK2'), tmp_df.value.substr(367,  2).alias('PUDWCK3'), tmp_df.value.substr(369,  2).alias('PUDWCK4'), tmp_df.value.substr(371,  2).alias('PUDWCK5'), tmp_df.value.substr(373,  2).alias('PEJHWKO'), tmp_df.value.substr(375,  2).alias('PUJHDP1O'), tmp_df.value.substr(377,  2).alias('PEJHRSN'), tmp_df.value.substr(379,  2).alias('PEJHWANT'), tmp_df.value.substr(381,  2).alias('PUJHCK1'), tmp_df.value.substr(383,  2).alias('PUJHCK2'), tmp_df.value.substr(385,  2).alias('PRABSREA'), tmp_df.value.substr(387,  2).alias('PRCIVLF'), tmp_df.value.substr(389,  2).alias('PRDISC'), tmp_df.value.substr(391,  2).alias('PREMPHRS'), tmp_df.value.substr(393,  2).alias('PREMPNOT'), tmp_df.value.substr(395,  2).alias('PREXPLF'), tmp_df.value.substr(397,  2).alias('PRFTLF'), tmp_df.value.substr(399,  2).alias('PRHRUSL'), tmp_df.value.substr(401,  2).alias('PRJOBSEA'), tmp_df.value.substr(403,  2).alias('PRPTHRS'), tmp_df.value.substr(405,  2).alias('PRPTREA'), tmp_df.value.substr(407,  3).alias('PRUNEDUR'), tmp_df.value.substr(410,  2).alias('FILLER410'), tmp_df.value.substr(412,  2).alias('PRUNTYPE'), tmp_df.value.substr(414,  2).alias('PRWKSCH'), tmp_df.value.substr(416,  2).alias('PRWKSTAT'), tmp_df.value.substr(418,  2).alias('PRWNTJOB'), tmp_df.value.substr(420,  2).alias('PUJHCK3'), tmp_df.value.substr(422,  2).alias('PUJHCK4'), tmp_df.value.substr(424,  2).alias('PUJHCK5'), tmp_df.value.substr(426,  2).alias('PUIODP1'), tmp_df.value.substr(428,  2).alias('PUIODP2'), tmp_df.value.substr(430,  2).alias('PUIODP3'), tmp_df.value.substr(432,  2).alias('PEIO1COW'), tmp_df.value.substr(434,  2).alias('PUIO1MFG'), tmp_df.value.substr(436,  6).alias('PADDING436'), tmp_df.value.substr(442,  2).alias('PEIO2COW'), tmp_df.value.substr(444,  2).alias('PUIO2MFG'), tmp_df.value.substr(446,  6).alias('PADDING446'), tmp_df.value.substr(452,  2).alias('PUIOCK1'), tmp_df.value.substr(454,  2).alias('PUIOCK2'), tmp_df.value.substr(456,  2).alias('PUIOCK3'), tmp_df.value.substr(458,  2).alias('PRIOELG'), tmp_df.value.substr(460,  2).alias('PRAGNA'), tmp_df.value.substr(462,  2).alias('PRCOW1'), tmp_df.value.substr(464,  2).alias('PRCOW2'), tmp_df.value.substr(466,  2).alias('PRCOWPG'), tmp_df.value.substr(468,  2).alias('PRDTCOW1'), tmp_df.value.substr(470,  2).alias('PRDTCOW2'), tmp_df.value.substr(472,  2).alias('PRDTIND1'), tmp_df.value.substr(474,  2).alias('PRDTIND2'), tmp_df.value.substr(476,  2).alias('PRDTOCC1'), tmp_df.value.substr(478,  2).alias('PRDTOCC2'), tmp_df.value.substr(482,  2).alias('PRMJIND1'), tmp_df.value.substr(484,  2).alias('PRMJIND2'), tmp_df.value.substr(486,  2).alias('PRMJOCC1'), tmp_df.value.substr(488,  2).alias('PRMJOCC2'), tmp_df.value.substr(490,  2).alias('PRMJOCGR'), tmp_df.value.substr(494,  2).alias('PRNAGWS'), tmp_df.value.substr(496,  2).alias('PRSJMJ'), tmp_df.value.substr(498,  2).alias('PRERELG'), tmp_df.value.substr(500,  2).alias('PEERNUOT'), tmp_df.value.substr(502,  2).alias('PEERNPER'), tmp_df.value.substr(504,  2).alias('PEERNRT'), tmp_df.value.substr(506,  2).alias('PEERNHRY'), tmp_df.value.substr(508,  4).alias('PUERNH1C'), tmp_df.value.substr(512,  4).alias('PEERNH2'), tmp_df.value.substr(516,  4).alias('PEERNH1O'), tmp_df.value.substr(520,  4).alias('PRERNHLY'), tmp_df.value.substr(524,  1).alias('PTHR'), tmp_df.value.substr(525,  2).alias('PEERNHRO'), tmp_df.value.substr(527,  8).alias('PRERNWA'), tmp_df.value.substr(535,  1).alias('PTWK'), tmp_df.value.substr(536,  4).alias('FILLER536'), tmp_df.value.substr(540,  8).alias('PEERN'), tmp_df.value.substr(548,  8).alias('PUERN2'), tmp_df.value.substr(556,  1).alias('PTOT'), tmp_df.value.substr(557,  2).alias('FILLER557'), tmp_df.value.substr(559,  2).alias('PEERNWKP'), tmp_df.value.substr(561,  2).alias('PEERNLAB'), tmp_df.value.substr(563,  2).alias('PEERNCOV'), tmp_df.value.substr(565,  2).alias('PENLFJH'), tmp_df.value.substr(567,  2).alias('PENLFRET'), tmp_df.value.substr(569,  2).alias('PENLFACT'), tmp_df.value.substr(571,  2).alias('PUNLFCK1'), tmp_df.value.substr(573,  2).alias('PUNLFCK2'), tmp_df.value.substr(575,  2).alias('PESCHENR'), tmp_df.value.substr(577,  2).alias('PESCHFT'), tmp_df.value.substr(579,  2).alias('PESCHLVL'), tmp_df.value.substr(581,  2).alias('PRNLFSCH'), tmp_df.value.substr(583, 10).alias('PWFMWGT'), tmp_df.value.substr(593, 10).alias('PWLGWGT'), tmp_df.value.substr(603, 10).alias('PWORWGT'), tmp_df.value.substr(613, 10).alias('PWSSWGT'), tmp_df.value.substr(623, 10).alias('PWVETWGT'), tmp_df.value.substr(633,  2).alias('PRCHLD'), tmp_df.value.substr(635,  2).alias('PRNMCHLD'), tmp_df.value.substr(637,  2).alias('FILLER637'), tmp_df.value.substr(639,  2).alias('PRWERNAL'), tmp_df.value.substr(641,  2).alias('PRHERNAL'), tmp_df.value.substr(643,  2).alias('HXTENURE'), tmp_df.value.substr(645,  2).alias('HXHOUSUT'), tmp_df.value.substr(647,  2).alias('HXTELHHD'), tmp_df.value.substr(649,  2).alias('HXTELAVL'), tmp_df.value.substr(651,  2).alias('HXPHONEO'), tmp_df.value.substr(653,  2).alias('PXINUSYR'), tmp_df.value.substr(657,  2).alias('PXPARENT'), tmp_df.value.substr(659,  2).alias('PXAGE'), tmp_df.value.substr(661,  2).alias('PXMARITL'), tmp_df.value.substr(663,  2).alias('PXSPOUSE'), tmp_df.value.substr(665,  2).alias('PXSEX'), tmp_df.value.substr(669,  2).alias('PXAFNOW'), tmp_df.value.substr(675,  2).alias('PXNATVTY'), tmp_df.value.substr(677,  2).alias('PXMNTVTY'), tmp_df.value.substr(679,  2).alias('PXFNTVTY'), tmp_df.value.substr(681,  2).alias('FILLER681'), tmp_df.value.substr(711,  2).alias('PXHRACT1'), tmp_df.value.substr(713,  2).alias('PXHRACT2'), tmp_df.value.substr(725,  2).alias('PXLAYDUR'), tmp_df.value.substr(735,  2).alias('PXLKLL2O'), tmp_df.value.substr(737,  2).alias('PXLKLWO'), tmp_df.value.substr(745,  2).alias('PXDWRSN'), tmp_df.value.substr(757,  2).alias('PXDWAVR'), tmp_df.value.substr(763,  2).alias('PXJHWANT'), tmp_df.value.substr(787,  4).alias('FILLER787'), tmp_df.value.substr(795,  2).alias('PXERNHRY'), tmp_df.value.substr(799,  2).alias('PXERNLAB'), tmp_df.value.substr(801,  2).alias('PXERNCOV'), tmp_df.value.substr(803,  2).alias('PXNLFJH'), tmp_df.value.substr(805,  2).alias('PXNLFRET'), tmp_df.value.substr(807,  2).alias('PXNLFACT'), tmp_df.value.substr(809,  2).alias('PXSCHENR'), tmp_df.value.substr(811,  2).alias('PXSCHFT'), tmp_df.value.substr(813,  2).alias('PXSCHLVL'), tmp_df.value.substr(815,  5).alias('QSTNUM'), tmp_df.value.substr(820,  2).alias('OCCURNUM'), tmp_df.value.substr(822,  2).alias('PEDIPGED'), tmp_df.value.substr(824,  2).alias('PEHGCOMP'), tmp_df.value.substr(826,  2).alias('PECYC'), tmp_df.value.substr(828,  2).alias('PEGRPROF'), tmp_df.value.substr(830,  2).alias('PEGR6COR'), tmp_df.value.substr(832,  2).alias('PEMS123'), tmp_df.value.substr(834,  2).alias('PXDIPGED'), tmp_df.value.substr(836,  2).alias('PXHGCOMP'), tmp_df.value.substr(838,  2).alias('PXCYC'), tmp_df.value.substr(840,  2).alias('PXGRPROF'), tmp_df.value.substr(842,  2).alias('PXGR6COR'), tmp_df.value.substr(844,  2).alias('PXMS123'), tmp_df.value.substr(846, 10).alias('PWCMPWGT'), tmp_df.value.substr(856,  4).alias('PEIO1ICD'), tmp_df.value.substr(860,  4).alias('PEIO1OCD'), tmp_df.value.substr(864,  4).alias('PEIO2ICD'), tmp_df.value.substr(868,  4).alias('PEIO2OCD'), tmp_df.value.substr(872,  2).alias('PRIMIND1'), tmp_df.value.substr(874,  2).alias('PRIMIND2'), tmp_df.value.substr(876,  2).alias('PEAFWHN1'), tmp_df.value.substr(878,  2).alias('PEAFWHN2'), tmp_df.value.substr(880,  2).alias('PEAFWHN3'), tmp_df.value.substr(882,  2).alias('PEAFWHN4'), tmp_df.value.substr(884,  2).alias('PXAFEVER'), tmp_df.value.substr(886,  2).alias('PELNDAD'), tmp_df.value.substr(888,  2).alias('PELNMOM'), tmp_df.value.substr(890,  2).alias('PEDADTYP'), tmp_df.value.substr(892,  2).alias('PEMOMTYP'), tmp_df.value.substr(894,  2).alias('PECOHAB'), tmp_df.value.substr(896,  2).alias('PXLNDAD'), tmp_df.value.substr(898,  2).alias('PXLNMOM'), tmp_df.value.substr(900,  2).alias('PXDADTYP'), tmp_df.value.substr(902,  2).alias('PXMOMTYP'), tmp_df.value.substr(904,  2).alias('PXCOHAB'), tmp_df.value.substr(906,  2).alias('PEDISEAR'), tmp_df.value.substr(912,  2).alias('PEDISPHY'), tmp_df.value.substr(916,  2).alias('PEDISOUT'), tmp_df.value.substr(918,  2).alias('PRDISFLG'), tmp_df.value.substr(920,  2).alias('PXDISEAR'), tmp_df.value.substr(922,  2).alias('PXDISEYE'), tmp_df.value.substr(924,  2).alias('PXDISREM'), tmp_df.value.substr(926,  2).alias('PXDISPHY'), tmp_df.value.substr(928,  2).alias('PXDISDRS'), tmp_df.value.substr(930,  2).alias('PXDISOUT'), tmp_df.value.substr(932,  2).alias('HXFAMINC'), tmp_df.value.substr(934, 17).alias('FILLER934'))